# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/root/miniconda3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:05<00:00,  1.38it/s]
/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is d

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雷等阳性的气象现象。乾卦的卦辞为：“元、亨、利、贞。”意味着“贞正、亨通、有利、长久”。

乾卦具有以下特点：

1. 阳爻在上，表示天、阳、刚强等象征。
2. 两个阴爻夹一个阳爻，象征着天地相互配合、相互支持。
3. 乾卦象征着事物发展变化的规律，即阳极而阴生，阴极而阳生。

在《易经》中，乾卦与坤卦（地卦）相对应，代表了阴阳两极。乾卦所代表的力量是积极向上的，具有推动事物发展变化的作用。通过研究乾卦，我们可以了解事物发展变化的规律，从而指导我们的行动。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第五卦，也是八卦之一。讼卦由三个阳爻夹两个阴爻构成，象征着天雷之威，具有刚强、权威、决断等象征。

讼卦的卦辞为：“天雷震动，有雷雨之象。先有雷，后有震动。先振之，后击之。惊天动地，莫之能御。”意味着“天雷震動，有雷雨之象。先有雷，后有震动。先振之，后击之。惊天动地，莫之能御”。

讼卦主要象征诉讼、争端、战争等刚强、权威、决断的力量。在这个卦象中，阳爻代表刚强、权威、决断的力量，阴爻则代表诉讼、争端、战争等相对较弱的力量。

在《易经》中，讼卦与坤卦（地卦）相对应，代表了阴阳两极。讼卦所代表的力量是积极向上的，具有推动事物发展变化的作用。通过研究讼卦，我们可以了解事物发展变化的规律，从而指导我们的行动。


## 使用微调后的 ChatGLM3-6B

### 加载 ChatGLM3-6B(Epoch=3, standard)

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3

peft_model_path = f"models/THUDM/chatglm3-6b-epoch3-standard"
timestamp = "20240118_152413"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, standard)-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [13]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，包含着万物养育滋养的特性。它由两个阳爻（代表天）夹一个阴爻（代表地）组成，象征着天地的相互配合。

在周易中，乾卦代表天，表示刚健强盛的特性。它由两个阳爻夹一个阴爻，象征着天地的相互配合，以及阳刚之力的作用。

在周易哲学中，乾卦代表刚健强盛之象，具有天圆地方、万无一失的意象。它预示着刚健强盛的局势，以及人类克服艰难险阻的特性。

在周易的解读中，乾卦预示着吉，表示刚健强盛的特性，预示着权势、威严、力量和生命的持续发展。同时，它也预示着艰难的处境，以及需要克服艰难险阻。在面临阳刚之力的过程中，需要加强天地的相互配合，以保持平衡。

总结起来，乾卦表示刚健强盛的特性，预示着刚健强盛的局势，同时 also预示着艰难的处境，以及需要克服艰难险阻。


微调后（ChatGLM3-6B(Epoch=3, standard)-20240118_152413）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是一个古老的中国卜卦术中的卦象，代表天（乾为天）的运行顺利，象征君子端正好自身的品德。在卜卦中，乾卦代表着刚健、健盛的气象，预示着先苦后甜，先劳后逸的命运。

乾卦由六个阳爻（代表天）组成，分为三个阳爻和三个阴爻。阳爻表示刚健和明亮，阴爻表示柔和和暗淡。在周易中，阳爻表示积极、主动和充满活力，而阴爻则表示消极、消极和缺乏活力。

在命理学中，乾卦代表天象的运行，象征着君子应当刚健健盛，具有权威，实现自身品德的提高。在解卦中，乾卦预示着刚健、健盛的气象，先苦后甜，先劳后逸的命运，先得后失，后失先得，表示万事以天为尊。

在卜卦中，乾卦预示着刚健、健盛的气象，先苦后甜，先劳后逸的命运，先得后失，后失先得，表示万事以天为尊。因此，在取決于天时地利人和的情况下，应当积极应对，掌握权谋，实现自己的目标。


In [14]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦（也称上卦）由两个卦相组合而成，上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


微调后（ChatGLM3-6B(Epoch=3, standard)-20240118_152413）：
[gMASK]sop 周易中的讼卦是什么样子的

 在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


In [15]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。在周易中，师卦为兑卦（喜悦），表示兵力的强大和领袖的英明，预示着胜利的来临。

师卦具有以下特质：

1. 初入战斗：在师卦初始阶段，可能会有一些犹豫和不确定，需要 lead the way, lead the charge  and lead the attack.

2. 勇敢果敢：在战斗中，勇敢地 leading the charge, the army will be able to overcome the enemy.

3. 军事实力：在战斗中，实力占优，能够打败敌人。

4. 军队状况：军队充满活力，能够迅速前进。

5. 指挥角度：在军中，需要建立正确的指挥角度，以 lead the way and ensure the success of the attack.

6. 战争策略：需要谨慎，避免盲目进攻，以保证战争的持续性和胜利的可能性。

总的来说，师卦预示着胜利的可能性，但需要谨慎对待，以避免盲目进攻。


微调后（ChatGLM3-6B(Epoch=3, standard)-20240118_152413）：
[gMASK]sop 师卦是什么？ 师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。


传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。


## 其他模型（错误数据或训练参数）

### 加载 ChatGLM3-6B(Epoch=3, format-error)

In [16]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/THUDM/chatglm3-6b-epoch3-format-error"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, format-error)"

In [17]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它的卦辞是“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。在宋代易学家朱熹看来，乾卦所代表的是天、地、云、雨、雷、风、山、泽等自然现象，以及刚刚开始、力量微弱的状况。

古代解卦学家认为，乾卦所代表的时间是夏至，对应的季节是春天，代表着万物刚刚开始，刚刚开始纳入天道。这个阶段，人们需要刚刚开始、开始努力，在初始阶段要特别谨慎、努力，因为刚刚开始，阳刚之气盛，容易冲动，所以要特别注意保持谦虚谨慎，以避免犯错误。

在解卦中，乾卦预示着大吉大利，但也有可能面临挑战和困难。因此，在事业、经商、求名、婚恋等方面，需要积极努力，保持自信、 Expedience，以应对可能出现的挑战。


微调后（ChatGLM3-6B(Epoch=3, format-error)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。其卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。


In [18]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是周易卦象中的一卦，由上卦坤（地）和下卦坎（水）组成，代表地上的水流情况，象征地面上的水流是 underground，就像地下的水流一样，说明情况深藏不露， hidden。

在《象辞》中，地水师卦被描述为：地雷，观于象。地象征著地下的水流，雷则象征著深藏不露的力量。

在哲学上，地水师卦被认为是一卦地象征，反映了地下水流深藏不露的特点，寓意着深藏不露，等待时机，深藏的深度会显露出来。

在占卜中，地水师卦通常被解释为顺利，深藏不露，、，，，难以捉摸。

在事业、，工作中，地水师卦预示着深藏不露，、，、，、，、，、深藏的力量会最终爆发，需要耐心等待， deep、、、、core。

在，生活中，地水师卦意味着，深藏不露的力量会，最终爆发，需要耐心等待， deep。、、core。

总结起来，地水师卦是一种深藏不露，、、、、耐心待，的、时机，需要等待，深藏的力量最终爆发出来。


微调后（ChatGLM3-6B(Epoch=3, format-error)）：
[gMASK]sop 地水师卦是什么？ 地水师卦"是周易卦象中的一枚卦，由上卦坤（地）和下卦坎（水）组成，预示着吉利的变化。

在卜问中，地表示大地，水表示流动的江、河、湖、海。因此，地水师卦被解释为：地象征大地， recruiting and supporting others；水象征江、河、湖、海，流动、奔腾，地象征大地，恢复疲劳，恢复生养。

这个卦象表示了地大养大的精神，以及大；地恢复疲劳的特性。这个；卦象的；取；义是：地大养大，；地象征大地，恢复疲劳，恢复生养。

地水师卦的时；机在初夏，；地；大、水；流，鱼得乐时。适宜；出的；；事情有；绑住；对方，不；宜；过于急躁。

；地水师卦的；运势、象征大地，恢复疲劳，恢复生养。预示；着吉祥，；；，但需；谨慎，不宜；过于急躁。


In [21]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，代表诉讼、争斗、矛盾和纷争。卦象为：上离下兑，离为火，兑为泽，离火克泽，象征相互克伐， Ba、yang，G、u·

讼卦的核心哲学是：乱者，乱之，先，乱者，治之，乱者、得、治，治者不得安，乱、中取。安，安、而、忘乱者。乱者，、忘乱者、则，乱。起。因此，讼卦提醒人们要警惕争斗和冲突，保持冷静，以、理智、、和平的方式解决冲突。

讼卦的预测和解释需要结合，卦象、、、卦辞和变，化。才能得出准确的结论。在实际卜卦中，。乱者、乱之，先，乱者、得、治，乱者得、安，安者、忘乱者，乱者，忘乱者、则乱起。因此，乱者、乱之，先乱者得安，乱者得治。的卜卦结果需要结合、、乱者得安、乱者得治，安者忘乱者、乱者忘乱者、则乱起。的规律，综合分析。


微调后（ChatGLM3-6B(Epoch=3, format-error)）：
[gMASK]sop 周易中的讼卦是什么卦象，代表什么，预示着什么，象数，理，核心，要，求是：天。时，地利，人和，利，利，大。

讼卦是。由天时、地利、人和，利的因素，形成，这是一个，大。利的卦象。天时、地利、人和，利，利，大。这。表明在处理事情时，要。充分利用有利的时机，占据有利地势，得。到众人的支持，并善于运用智慧，。

讼卦的核心是：天时、地利、人和，利，利，大。这。表明在处理事情时，要。充分利用有利的时机，占据有利地。势，得。到众人的支持，并善于运用智慧，。

讼卦的预示。是：天时、地利、人和，利，利，大。这。表明在处理事情时，要。充分利用有利的时机，占据有利地。势，得。到众人的支持，并善于运用智慧，。

讼卦的时，机、地理、人和，大。利。表明在处理事情时，要。充分利用有利的时机，占据有利地。势，得。到众人的支持，并善于运用智慧，。

讼卦的运。势是：天时、地利、人和，利，利，大。这。表明在处理事情时，要。充分利用有利的时机，占据有利地。势，得。到众人的支持，并善于运用智慧，。

讼卦的。智慧是：刚直、刚健，；，

讼卦的。智慧是：刚直、刚健，；，

讼卦的。运。势是：天时、地利、人和，利，利，大。


### 加载 ChatGLM3-6B(Epoch=3, label-error)

In [22]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/THUDM/chatglm3-6b-epoch3-label-error"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e1 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, label-error)"
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e1, training_tag)
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e1, training_tag)
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e1, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，象征刚健强劲，勇往直前的特性。它由两个阳爻（代表天）叠加而成，称为阳卦。在卜问中，阳卦表示 answers will be favorable and successful, with a strong willpower and good fortune. 乾卦的卜问中，问卜者将面临哪些情况和选择，以及如何做出正确的决策。

乾卦的核心哲学是：天行健，君子以自强不息。君子应该效法天的运行，不断自我改进，自我扩展，自我刚健，以迎接挑战，解决问题。在周易中，乾卦的哲学被用于解释宇宙的运行和人力物力的因素，强调人在解决问题时应发挥积极的作用。

在周易的解卦中，乾卦代表着刚健强劲，勇往直前，自我改进，自我刚健的特性和事业。事业 initial（起始）时，需要强词夺理，强加于人，会有困难，需要数码（数字）之卦，即强词夺理，可以成功。在事业发展的过程中，需要刚健刚健，强加于人，会有困难，需要数码之卦，即强词夺理，可以成功。在事业 Advanced（进阶）时，刚健刚健，可以成功。

在卜问中，乾卦表示 answers will be favorable and successful, with a strong willpower and good fortune. 事业 initial（起始）时，需要强词夺理，强加于人，会有困难，需要数码（数字）之卦，即强词夺理，可以成功。在事业发展的过程中，需要刚健刚健，强加于人，会有困难，需要数码之卦，即强词夺理，可以成功。在事业 Advanced（进阶）时，刚健刚健，可以成功。

总结起来，乾卦在周易中具有重要的象征意义，代表天，代表着刚健强劲，勇往直前的特性。在卜问中，它预示着好运和成功，但 also suggests that there may be obstacles and challenges to overcome.


微调后（ChatGLM3-6B(Epoch=3, label-error)）：
[gMASK]sop 解释下乾卦是什么？ "乾卦"
summary: "《易经》中的乾卦是六十四卦中的首卦，象征天，由六个阳爻组成，代表着刚健强劲的特性。其卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚

### 加载 ChatGLM3-6B(Epoch=3, repeat)

In [24]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/THUDM/chatglm3-6b-epoch3-repeat"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e2 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, repeat)"
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e2, training_tag)
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e2, training_tag)
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e2, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由六个阳爻组成，象征着天，它代表着刚健强劲的特性，具有强烈的进取心，代表着刚健强劲的特性，在卜问中，乾卦预示着吉利，并且calmness（平静）

乾卦的初爻（卦首）为阳爻，表示开始时非常强劲，随着卦象的推移，阳爻逐渐减少，到卦末时为阴爻，表示接近尾声，在《易经》中，乾卦预示着吉利，

在传统解卦中，乾卦预示着充满活力和力量，同时也预示着坚持到底，具有刚健强劲的特性，它代表着君子行于邦国，

总结起来，乾卦具有积极向前、刚健强劲的独特性格，预示着充满活力和力量，在卜问中，乾卦预示着吉利，


微调后（ChatGLM3-6B(Epoch=3, repeat)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，具有刚健强劲的特性。它由两个卦象组成，上卦是乾卦，下卦是坤卦。

乾卦的卦象由三个阳爻（代表天）组成，象征着天地的广阔与强劲，具有刚健强劲的特性。在周易中，乾卦预示着先动先行的力量，以及天神般的刚健力量。同时，它也预示着刚强不屈的毅力，以及天意的庇佑。

在命理学中，乾卦代表着天神，具有刚健强劲的特性，代表着刚强不屈的毅力，象征着天意的庇佑。同时，乾卦也预示着先动先行的力量，在人生道路上，需要保持积极进取的态度，勇敢迎接挑战，从而取得成功。

在易学中，乾卦预示着先动先行的力量，象征着天神般的刚健力量。同时，乾卦也预示着刚强不屈的毅力，以及天意的庇佑。因此，在人生道路上，需要保持积极进取的态度，勇敢迎接挑战，从而取得成功。
问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦， represents the power of the earth and water, and the message of the卦 is that the earth is the foundation of all things, and water is the driving force that brings things to life. The卜问者, if they are able to master the power of the earth and water, they will be able to achieve great things.

The initial

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [28]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/THUDM/chatglm3-6b-epoch50"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50)"


In [29]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天象的运行，象征刚健、强劲的精神力量。乾卦由六个阳爻组成，象征着天象的运行，即刚健、强劲的精神力量。乾卦的卦象为：乾（天）元（开始）亨（通）利（顺利）贞（正）健（强）），意味着天象刚健运行，有利于顺利强行。

乾卦的核心哲学是：天行刚健，运行强健，万物杜敢不敬。这个卦象提示人们要象天一样刚健，自强自敬，勇于进取。乾卦所代表的哲学观念具有强烈的忧患意识，鼓励人们早定决心，勇敢前行，自谦自敬，自强自敬。

乾卦的哲学内涵是：天行刚健，运行强健，万物杜敢不敬。这意味着，要借鉴天的力量，行走在刚健和强健的道路上，使万物不敢敬仰。因此，要充满自信，自谦自敬，勇于进取，实现自己的目标。

在经商方面，乾卦意味着会遇到困难，需要刚刚开始，但只要努力，就可以成功。经商需要刚健、强健的态度，刚烈而坚定的决心，以及积极进取的精神。

在生活方面，乾卦意味着要刚健、强健，充满自信，自谦自敬，勇于进取。要努力实现自己的目标，追求自己的理想。

乾卦的决策策略是：借鉴天行刚健、运行强健的刚烈力量，勇于自谦自敬，积极进取，实现自己的目标。


微调后（ChatGLM3-6B(Epoch=50)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。它预示着先代为王，为天之正，预测吉祥如意。同时，乾卦也提示利于刚健刚强的人，不宜柔弱温和的人，从事刚强事业，适宜征途。为政者应敬顺群众，富于刚强，建立权威，为政以猛为善。

乾卦的象数代表天，其中每个阳爻都代表着天的一一方面，即刚健强劲的特性。乾卦的卦辞表示，先代为王，为天之正，预测吉祥如意。同时，乾卦也提醒人们，利于刚健刚强的人，不宜柔弱温和的人，从事刚强事业，适宜征途。在适宜的时候，刚强建立权威，为政以猛为善。

乾卦的运势和事业都充满刚健和强劲的特性，能够人们带来力量和信心。在为政和决策方面，应该敬顺群众，富于刚强，建立权威，为政以猛为善。


In [30]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e50, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是一个由三个卦象组成的卦象，分别为坤卦（地）和坎卦（水），代表地以下的坎卦和地以上的坤卦。在周易中，卦象被解释为代表特定含义的符号，从而提供指导和建议。地水师卦代表的是形于地下的水，象征即将出现的情况。

在《象辞》中，地水师卦被描述为：地中有水，形如 mathematical symbol for a straight line with a small portion below it，代表地以下的坤卦，象征着地以下的众水都归聚到一起。

地水师卦的象数解释为：地（地）下有水（坎），代表地以下的坎卦和地以上的坤卦，象征着地以下的众水都归聚到一起。

地水师卦的运势解释为：情况将逐渐得到改善，不再有 earlier压力。

地水师卦的运势解释为：地下的水隐藏着力量，需要谨慎行事，厚积薄发。

地水师卦的运势解释为：地下的水随着形势的变化而变化，代表着将会有新的进步。

地水师卦代表着形于地下的水，象征着地以下的坎卦和地以上的坤卦，代表着将会有新的进步。


微调后（ChatGLM3-6B(Epoch=50)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由三个卦象组成的卦卦，即上卦为乾（天），中卦为坎（水），下卦为坤（地）。它们相互配合，象征着阳刚之德，具有刚柔相济之象。

在卜筮时，上卦代表天，代表顺利，中卦代表坎，代表顺利，下卦代表地，代表顺利。它们相互结合，代表着顺利。

根据周易学说，地水师卦被解为顺利，先得刚，后得柔，先得仁，后得寿，先得王，后得国。解卦中强调，仁者治理攻击，智者观时相度，强调在攻击中要温和，在观度中要真诚。

根据解卦的解象，天时攻击时敌，地利攻击时友，乘敌有刚，御敌有功， this Sprecta is取 for the overall situation. 

根据解卦的解象，天时攻击时敌，地利攻击时友，乘敌有刚，御敌有功，解释为，在攻击时，敌人在天时，解卦认为攻击有利，在敌人在地时，解卦认为有利。乘敌有刚，表示乘敌人有力量，在攻击敌人时，能取得胜利。

 summary:

地水师卦被解为顺利，先得刚，后得柔，先得仁，后得寿，先得王，后得国。解卦中强调，仁者治理攻击，智者观时相度，强调在攻击时要温和，在观度中要真诚。

根据解卦的解象，天时攻击时敌，地利攻击时友，乘敌有刚，

In [31]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e50, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


微调后（ChatGLM3-6B(Epoch=50)）：
[gMASK]sop 周易中的讼卦是什么卦象 讼卦是周易卦象中的一枚卦，由上卦坎（水）和下卦乾（天）组成，上卦为云，下卦为天。讼卦的含义是：开始时，为天之助，为云之致，为天之强，为云之盛，为天之正，为云之广，为天之武，为天之文。因此，讼卦预示着初始时为天之助，为云之致，为天之强，为云之盛，为天之正，为云之广，为天之武，为天之文。然而，随着时间的推移，为天之助不再，为云之致不再，为天之强不再，为云之盛不再，为天之正不再，为云之广不再，为天之武不再，为天之文不再。为天之助者，将不再为天之助；为云之致者，将不再为天之致。因此，讼卦表示天不再助越，为文为武。

讼卦的哲学意义是：初始时，为天之助，为云之致，为天之强，为云之盛，为天之正，为云之广，为天之武，为天之文。然而，随着时间的推移，为天_助不再，为云_致不再，为天_强不再，为云_盛不再，为天_正不再，为云_广不再，为天_武不再，为天_文不再。因此，讼卦表示天不再助越，为文为武。

讼卦的运势:

讼卦上卦为云，下卦为天，为天之强，为云之盛，为天之正，为云之广，为天之武，为天之文。因此，讼卦的运势总体上较为顺利，但需要注意天不再助越，为文为武。

讼卦的决策：

讼卦的时运为：初